<a href="https://colab.research.google.com/github/anjali-rgpt/Autocomplete/blob/master/Neural_Model_Using_Single_LSTM_Unit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
import random, sys

Using TensorFlow backend.


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 0s 0us/step


In [3]:
# import data 
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# text = text[:round(len(text)/2)]
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
text = text[:1000]

chars = sorted(list(set(text)))
print('Vocabulary:', len(chars))
# char_indices = dict((c, i) for i, c in enumerate(chars))
# indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i: i + maxlen])
  next_chars.append(text[i + maxlen])
print('Number of sequences: ', len(sentences))

# x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
# y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
# for i, sentence in enumerate(sentences):
#   for t, char in enumerate(sentence):
#       x[i, t, char_indices[char]] = 1
#   y[i, char_indices[next_chars[i]]] = 1
# print('Vectorised!')

Vocabulary: 46
Number of sequences:  320


In [5]:
from sklearn.model_selection import train_test_split

vocab_size = len(chars)

X_train, X_test , y_train, y_test = train_test_split(sentences, next_chars , test_size = 0.2)

X_train = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_train]  #remove punctuation
X_test = [one_hot(d, vocab_size,filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~',lower=True, split=' ') for d in X_test]

X_train = pad_sequences(X_train, maxlen=maxlen, padding='pre')
X_test = pad_sequences(X_test, maxlen=maxlen, padding='pre')

In [6]:
y_train = np.asarray(y_train)
y_test = np.asarray(y_test)

In [7]:
print('Build model...')
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Embedding(len(chars), 1, input_length=maxlen))
model.add(tf.keras.layers.LSTM(8, input_shape=(maxlen, len(chars))))
model.add(tf.keras.layers.Dense(len(chars), activation='softmax'))

# optimizer = tf.keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')

Build model...


In [ ]:
model.fit(X_train, y_train,batch_size=64,epochs=20)

Epoch 1/20


In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)





In [ ]:
def display():
  start_index = random.randint(0, len(text) - maxlen - 1)
  for diversity in [0.2, 0.5, 1.0, 1.2]:
    generated = ''
    sentence = text[start_index: start_index + maxlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(50):
      x_pred = np.zeros((1, maxlen, len(chars)))
      for t, char in enumerate(sentence):
        x_pred[0, t, char_indices[char]] = 1.
        preds = model.predict(x_pred, verbose=0)[0]
        next_index = sample(preds, diversity)
        next_char = indices_char[next_index]
        sentence = sentence[1:] + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
      print()

In [ ]:
display()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file='textgenmodel.png')